In [1]:
from numpy.random import seed
seed(1)

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import tensorflow
tensorflow.keras.__version__

'2.4.0'

In [3]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
data =  df['koi_disposition']!='CANDIDATE'
df = df[data]
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

In [4]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_depth','koi_insol', 'koi_model_snr', 'koi_prad', 'koi_teq', 'koi_period','koi_steff']]
X = selected_features

In [5]:
y = df[['koi_disposition']] 

In [6]:
print(X.shape, y.shape)

(5304, 7) (5304, 1)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [9]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=7))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [18]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               800       
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 202       
Total params: 41,402
Trainable params: 41,402
Non-trainable params: 0
__________________________________________________

In [20]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
125/125 - 0s - loss: 0.5485 - accuracy: 0.6953
Epoch 2/60
125/125 - 0s - loss: 0.4635 - accuracy: 0.7740
Epoch 3/60
125/125 - 0s - loss: 0.4397 - accuracy: 0.7851
Epoch 4/60
125/125 - 0s - loss: 0.4347 - accuracy: 0.7873
Epoch 5/60
125/125 - 0s - loss: 0.4271 - accuracy: 0.7939
Epoch 6/60
125/125 - 0s - loss: 0.4235 - accuracy: 0.7914
Epoch 7/60
125/125 - 0s - loss: 0.4231 - accuracy: 0.7951
Epoch 8/60
125/125 - 0s - loss: 0.4142 - accuracy: 0.8009
Epoch 9/60
125/125 - 0s - loss: 0.4143 - accuracy: 0.7981
Epoch 10/60
125/125 - 0s - loss: 0.4163 - accuracy: 0.7971
Epoch 11/60
125/125 - 0s - loss: 0.4096 - accuracy: 0.8014
Epoch 12/60
125/125 - 0s - loss: 0.4115 - accuracy: 0.8022
Epoch 13/60
125/125 - 0s - loss: 0.4123 - accuracy: 0.7996
Epoch 14/60
125/125 - 0s - loss: 0.4063 - accuracy: 0.8024
Epoch 15/60
125/125 - 0s - loss: 0.4099 - accuracy: 0.8014
Epoch 16/60
125/125 - 0s - loss: 0.4080 - accuracy: 0.8002
Epoch 17/60
125/125 - 0s - loss: 0.4071 - accuracy: 0.8022
Epoch 

In [21]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

42/42 - 0s - loss: 0.3872 - accuracy: 0.8100
Normal Neural Network - Loss: 0.38722512125968933, Accuracy: 0.8099547624588013


In [23]:
encoded_predictions_all = model.predict_classes(X_test_scaled)
prediction_labels_all = label_encoder.inverse_transform(encoded_predictions_all)
prediction_labels_all

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array(['FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', ...,
       'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED'], dtype=object)

In [25]:
# Calculate classification report
target_names = ["CONFIRMED", "FALSE POSITIVE"]
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction_labels_all,
                            target_names=target_names))

                precision    recall  f1-score   support

     CONFIRMED       0.70      0.82      0.75       471
FALSE POSITIVE       0.89      0.80      0.85       855

      accuracy                           0.81      1326
     macro avg       0.79      0.81      0.80      1326
  weighted avg       0.82      0.81      0.81      1326



In [32]:
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
cm = confusion_matrix(y_test,prediction_labels_all)
cm

array([[386,  85],
       [167, 688]], dtype=int64)